In [0]:
# install SNN-Toolbox
!pip install /content/drive/My\ Drive/GitHub/spiking-p3-gtnumbers/snn_toolbox

Processing ./drive/My Drive/GitHub/spiking-p3-gtnumbers/snn_toolbox
  Created wheel for snntoolbox: filename=snntoolbox-0.4.1-py2.py3-none-any.whl size=195127 sha256=e6206ed975f571e60dcbe65ebf73b24486c2e1bd2321d93e90580faa789418cb
  Stored in directory: /tmp/pip-ephem-wheel-cache-683zqj6e/wheels/a6/22/3b/c7bbca8a559a101b4cfb0bef50ae45a6e62a1cf48ae44ffcb8
Successfully built snntoolbox


In [0]:
%matplotlib notebook

from scipy.io import loadmat
import numpy as np
import keras
import logging
from keras.layers import (Conv2D, BatchNormalization, Dropout, AveragePooling2D,
Flatten, Dense, Input)
from keras.models import Sequential
from sklearn.model_selection import cross_val_score, ShuffleSplit
import tensorflow as tf
import os
import datetime
import snntoolbox
from snntoolbox.utils.utils import import_configparser
from snntoolbox.bin.run import main
from sklearn.model_selection import train_test_split

# set logging level
logging.basicConfig(level=logging.DEBUG)
# for replication
np.random.seed(0)
tf.random.set_seed(1)
# set up working directory to store results
path_wd = os.path.abspath(
    os.path.join('/content/drive/My Drive/', 'temp',
                 str(datetime.datetime.now())))
os.makedirs(path_wd)
model_name = 'gtn_model'
model_ext = '.h5'
# load data
datapath = '/content/drive/My Drive/VarekaGTNEpochs.mat'
data = loadmat(datapath)
target = data['allTargetData']
non_target = data['allNonTargetData']
logging.info('Loaded data from file {}.'.format(datapath))

Using TensorFlow backend.
INFO:root:Loaded data from file /content/drive/My Drive/VarekaGTNEpochs.mat.


In [0]:
# preprocess data
def discard_damaged(features, labels):
    threshold = 100.0
    accept_features = []
    accept_labels = []
    for i in range(features.shape[0]):
        discard = False
        # check max values of all channels
        if np.max(np.abs(features[i])) > threshold:
            discard = True
        if not discard:
            accept_features.append(features[i])
            accept_labels.append(labels[i])
    logging.info('Discarded {}% of samples.'.format(
        (1 - len(accept_features) / features.shape[0]) * 100))
    features = np.array(accept_features)
    labels = np.array(accept_labels)
    return features, labels

features = np.concatenate((target, non_target), axis=0)
target_labels = np.tile(np.array([1, 0]), (target.shape[0], 1))
non_target_labels = np.tile(np.array([0, 1]), (non_target.shape[0], 1))
labels = np.vstack((target_labels, non_target_labels))
(features, labels) = discard_damaged(features, labels)
features = np.expand_dims(features, -1)
logging.debug('preprocessed features({}): {})'.format(features.shape, features))
logging.debug('preprocessed labels({}): {})'.format(labels.shape, labels))

INFO:root:Discarded 30.315643426985783% of samples.
DEBUG:root:preprocessed features((8036, 3, 1200, 1)): [[[[ -0.77055806]
   [ -0.76216829]
   [ -0.75329262]
   ...
   [ -0.40571865]
   [ -0.68341094]
   [ -1.0573101 ]]

  [[ -4.92086267]
   [ -4.85863543]
   [ -4.80766249]
   ...
   [ -3.9121232 ]
   [ -3.8644805 ]
   [ -3.93403864]]

  [[  3.94080687]
   [  3.87810278]
   [  3.78919363]
   ...
   [  0.9730792 ]
   [  1.04828787]
   [  1.00901747]]]


 [[[  4.72857332]
   [  4.79228735]
   [  4.86664867]
   ...
   [ -7.4045186 ]
   [ -7.42412901]
   [ -7.48302507]]

  [[ 12.06944942]
   [ 11.96268559]
   [ 11.86560726]
   ...
   [-21.91578674]
   [-22.24193764]
   [-22.54764175]]

  [[ 15.44363117]
   [ 15.17852116]
   [ 14.91344929]
   ...
   [-15.98517323]
   [-16.30454826]
   [-16.64167595]]]


 [[[ -2.2145133 ]
   [ -1.49968874]
   [ -0.77021754]
   ...
   [-17.29611206]
   [-17.57557297]
   [-17.91339111]]

  [[ -7.53343868]
   [ -7.34118509]
   [ -7.12659883]
   ...
   [ -3.52

In [0]:
# Create a config file for SNN Toolbox.
configparser = import_configparser()
config = configparser.ConfigParser()
config['paths'] = {
    'path_wd': path_wd,  # Path to model.
    'dataset_path': path_wd,  # Path to dataset.
    'filename_ann': model_name  # Name of input model.
}

config['tools'] = {
    'evaluate_ann': True,  # Test ANN on dataset before conversion.
    'normalize': True  # Normalize weights for full dynamic range.
}

config['simulation'] = {
    'simulator': 'INI',  # Chooses execution backend of SNN toolbox.
    'duration': 70,  # Number of time steps to run each sample.
    'num_to_test': 2009,  # How many test samples to run.
    'batch_size': 49,  # Batch size for simulation.
}

config['output'] = {
    # 'plot_vars': {
    #     'spiketrains',
    #     'spikerates',
    #     'activations',
    #     'correlation',
    #     'v_mem',
    #     'error_t'}
}

# Store config file.
config_path = os.path.join(path_wd, 'config')
with open(config_path, 'w') as configfile:
    config.write(configfile)
logging.info('SNN-Toolbox config file written to {}'.format(config_path))

INFO:root:SNN-Toolbox config file written to /content/drive/My Drive/temp/2020-04-27 19:40:19.386830/config


In [0]:
# Prepare Keras model
def create_model():
  model = Sequential()
  model.add(Conv2D(filters=6, kernel_size=(3, 3),
    activation=keras.activations.elu, input_shape=(3, 1200, 1)))
  model.add(BatchNormalization())
  model.add(Dropout(0.5, seed=0))
  model.add(AveragePooling2D(pool_size=(1, 8)))
  model.add(Flatten())
  model.add(Dense(100, activation=keras.activations.elu))
  model.add(BatchNormalization())
  model.add(Dropout(0.5, seed=0))
  model.add(Dense(2, activation=keras.activations.softmax))
  model.compile(
      optimizer=keras.optimizers.Adam(), loss=keras.losses.BinaryCrossentropy(),
      metrics=['binary_accuracy'])
  return model

In [0]:
x_train, x_test, y_train, y_test = train_test_split(
            features, labels, test_size=0.25, random_state=0, shuffle=True
        )
# save test data for SNN Toolbox
logging.debug('Saving test data...')
np.savez_compressed(os.path.join(path_wd, 'x_test'), x_test)
np.savez_compressed(os.path.join(path_wd, 'y_test'), y_test)

shuffle_split = ShuffleSplit(
    n_splits=30,
    test_size=0.25,
    random_state=0
)
es = keras.callbacks.callbacks.EarlyStopping(
    patience=5, verbose=1, restore_best_weights=True)
counter = 0
validation_metrics = []
test_metrics = []

for train, validation in shuffle_split.split(x_train):
    print('Iteration {}'.format(counter))
    ann_model = create_model()
    val_metrics[counter] = ann_model.fit(
        x=x_train[train], 
        y=y_train[train],
        callbacks=[es],
        shuffle=True,
        validation_data=(x_train[validation], y_train[validation])
      )
    test_metrics[counter] = ann_model.evaluate(x_test, y_test)
    logging.info('Saving ANN model...')
    ann_model.save(os.path.join(
        path_wd, ''.join([model_name, model_ext])))
    np.savez_compressed(os.path.join(path_wd, 'x_norm'), (x_train[train])[::10])
    logging.info('Converting ANN to SNN...')
    results[counter] = main(config_path)
    counter += 1


DEBUG:root:Saving test data...


Train on 4520 samples, validate on 1507 samples
Epoch 1/1
2009/2009 [==============================] - 1s 425us/step


INFO:root:Saving ANN model...
INFO:root:Converting ANN to SNN...


Initializing INI simulator...

Loading data set from '.npz' files in /content/drive/My Drive/temp/2020-04-27 19:40:19.386830.

Model: "sequential_1"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_1 (Conv2D)            (None, 1, 1198, 6)        60        
_________________________________________________________________
batch_normalization_1 (Batch (None, 1, 1198, 6)        24        
_________________________________________________________________
dropout_1 (Dropout)          (None, 1, 1198, 6)        0         
_________________________________________________________________
average_pooling2d_1 (Average (None, 1, 149, 6)         0         
_________________________________________________________________
flatten_1 (Flatten)          (None, 894)               0         
_________________________________________________________________
dense_1 (Dense)              (None, 100)               8950

<IPython.core.display.Javascript object>

...

Initializing INI simulator...

Loading scale factors from disk instead of recalculating.
Loading data set from '.npz' files in /content/drive/My Drive/temp/2020-04-27 19:40:19.386830.

Model: "sequential_25"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_25 (Conv2D)           (None, 1, 1198, 6)        60        
_________________________________________________________________
batch_normalization_49 (Batc (None, 1, 1198, 6)        24        
_________________________________________________________________
dropout_49 (Dropout)         (None, 1, 1198, 6)        0         
_________________________________________________________________
average_pooling2d_25 (Averag (None, 1, 149, 6)         0         
_________________________________________________________________
flatten_25 (Flatten)         (None, 894)               0         
_________________________________________________________________


INFO:root:Saving ANN model...
INFO:root:Converting ANN to SNN...


Initializing INI simulator...

Loading scale factors from disk instead of recalculating.
Loading data set from '.npz' files in /content/drive/My Drive/temp/2020-04-27 19:40:19.386830.

Model: "sequential_26"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_26 (Conv2D)           (None, 1, 1198, 6)        60        
_________________________________________________________________
batch_normalization_51 (Batc (None, 1, 1198, 6)        24        
_________________________________________________________________
dropout_51 (Dropout)         (None, 1, 1198, 6)        0         
_________________________________________________________________
average_pooling2d_26 (Averag (None, 1, 149, 6)         0         
_________________________________________________________________
flatten_26 (Flatten)         (None, 894)               0         
_________________________________________________________________


INFO:root:Saving ANN model...
INFO:root:Converting ANN to SNN...


Initializing INI simulator...

Loading scale factors from disk instead of recalculating.
Loading data set from '.npz' files in /content/drive/My Drive/temp/2020-04-27 19:40:19.386830.

Model: "sequential_27"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_27 (Conv2D)           (None, 1, 1198, 6)        60        
_________________________________________________________________
batch_normalization_53 (Batc (None, 1, 1198, 6)        24        
_________________________________________________________________
dropout_53 (Dropout)         (None, 1, 1198, 6)        0         
_________________________________________________________________
average_pooling2d_27 (Averag (None, 1, 149, 6)         0         
_________________________________________________________________
flatten_27 (Flatten)         (None, 894)               0         
_________________________________________________________________
